## Import

In [1]:
import os
import subprocess
import html
import re

from tqdm import tqdm
import thaispellcheck
from html.parser import HTMLParser
from attacut import tokenize

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor

## List all pdf in folder

In [2]:
list_of_pdf = os.listdir('./pdf_example')
list_of_pdf = [x for x in list_of_pdf if x.endswith('.pdf')]
list_of_name = [list_of_pdf[num].split('.')[0]+'.'+list_of_pdf[num].split('.')[1]+'.'+list_of_pdf[num].split('.')[2] for num in range(len(list_of_pdf))]
list_of_name

['10-ก.พ.-2566+Academic_100223_115125',
 '11-ธ.ค.-2555+Academic_111212_150945',
 '11-ธ.ค.-2555+Academic_111212_152818',
 '11-ธ.ค.-2555+Academic_111212_154616',
 '11-ธ.ค.-2555+Academic_111212_154855',
 '3-ก.พ.-2565+Academic_110821_091216',
 '3-ต.ค.-2566+Academic_031023_105516',
 '4-มิ.ย.-2556+Academic_040613_100424',
 '4-มิ.ย.-2556+Academic_040613_100623',
 '9-ส.ค.-2566+Academic_130623_110407']

## Convert PDF to HTML with PDFBox ( java )

In [3]:
for num in range(len(list_of_name)):
    pdf_file = list_of_name[num]
    path_to_pdf_file = 'pdf_example/'+pdf_file+'.pdf'
    input_format = '--input='+path_to_pdf_file
    print(path_to_pdf_file)
    print(list_of_name[num])
    output_path = "./html_output/before_mapping"
    os.makedirs(output_path, exist_ok=True)
    # คำสั่งที่ต้องการรัน
    command = ['java', '-jar', 'pdfbox-app-3.0.2.jar', 'export:text', '-html', input_format, f'--output={output_path}/{list_of_name[num]}.html']

    # รันคำสั่งและเก็บผลลัพธ์
    result = subprocess.run(command, capture_output=True, text=True)

pdf_example/10-ก.พ.-2566+Academic_100223_115125.pdf
10-ก.พ.-2566+Academic_100223_115125
pdf_example/11-ธ.ค.-2555+Academic_111212_150945.pdf
11-ธ.ค.-2555+Academic_111212_150945
pdf_example/11-ธ.ค.-2555+Academic_111212_152818.pdf
11-ธ.ค.-2555+Academic_111212_152818


Exception in thread Thread-15:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte


pdf_example/11-ธ.ค.-2555+Academic_111212_154616.pdf
11-ธ.ค.-2555+Academic_111212_154616
pdf_example/11-ธ.ค.-2555+Academic_111212_154855.pdf
11-ธ.ค.-2555+Academic_111212_154855
pdf_example/3-ก.พ.-2565+Academic_110821_091216.pdf
3-ก.พ.-2565+Academic_110821_091216
pdf_example/3-ต.ค.-2566+Academic_031023_105516.pdf
3-ต.ค.-2566+Academic_031023_105516
pdf_example/4-มิ.ย.-2556+Academic_040613_100424.pdf
4-มิ.ย.-2556+Academic_040613_100424
pdf_example/4-มิ.ย.-2556+Academic_040613_100623.pdf
4-มิ.ย.-2556+Academic_040613_100623
pdf_example/9-ส.ค.-2566+Academic_130623_110407.pdf
9-ส.ค.-2566+Academic_130623_110407


Exception in thread Thread-29:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte


## Mapping some Error Code

In [3]:
pua = {
    '63233': '&#3636;', # 0xf701 Sara I
    '63234': '&#3637;', # 0xf702
    '63235': '&#3638;', # 0xf703
    '63236': '&#3639;', # 0xf704
    '63237': '&#3656;', # 0xf705
    '63238': '&#3657;', # 0xf706 Mai Tho (on Po Pla)
    '63242': '&#3656;', # 0xf70a Mai Ek
    '63243': '&#3657;', # 0xf70b Mai Tho
    '63246': '&#3660;', # 0xf70e Thantakat
    '63248': '&#3633;', # 0xf710  Mai Han Akhat (on Po Pla)
    '63250': '&#3655;', # 0xf712 Mai Tai Khu (on Po Pla)
    '63251': '&#3656;', # 0xf713
    '63252': '&#3657;', # 0xf714
    '63244': '&#3658;', # ๊
    '63247': '&#3597;', #ญ
    '61482': '&#42;',   #*
    '61591': '&#8226', #•
    '63240': '&#3657', # Mai Tho
    '63241': '&#3652', #Thantakat (on Po Pla)
    '65288': '&#40;', #(
    '65289': '&#41;', #)
    '61623': '&#8226', #•
    '65309': '&#61', #=
    '65293': '&#45', #-
    '65374': '&#126', #~
}

def thaiPUA(matchobj):
    return pua[matchobj.group(1)]

# p = re.compile(r'\&\#(\d{5,})\;')
p = re.compile(r'\&\#(6\d{4,})\;')

for num in range(len(list_of_name)):
    output_after_mapping = "./html_output/after_mapping/"
    os.makedirs(output_after_mapping, exist_ok=True)
    outputf = open(f'{output_after_mapping}/{list_of_name[num]}.html', 'w')
    inputf = open(f'{output_path}/{list_of_name[num]}.html', 'r')
    print(list_of_name[num])
    for line in inputf:
        text = p.sub(thaiPUA, line)
        
        outputf.writelines(html.unescape(text))
    inputf.close()
    outputf.close()

NameError: name 'output_path' is not defined

## Convert HTML to TXT

In [4]:
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

digits = {
    '๐': '0',
    '๑': '1',
    '๒': '2',
    '๓': '3',
    '๔': '4',
    '๕': '5',
    '๖': '6',
    '๗': '7',
    '๘': '8',
    '๙': '9'
}

def thaiDigits(matchobj):
    return digits[matchobj.group(0)]

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def tonemarkPos(matchobj):
    return matchobj.group(2) + matchobj.group(1)

def tonemarkSpace(matchobj):
    return matchobj.group(1) + matchobj.group(2)

def process_text(text):
    p2 = re.compile(r'([่้๊๋])([ัุูิีึื])') # <tonemark><vowel> -> <vowel><tonemark>
    p3 = re.compile(r'([ัุูิีึื])\s+([่้๊๋])') # <vowel><space><tonemark> -> <vowel><tonemark>
    p4 = re.compile(r'[ ]+$', re.MULTILINE) # <space>+ -> <space>
    p7 = re.compile(r'\s+(?=[ัุูิีึื])') # <space><vowel> -> <vowel>

    text = strip_tags(text)  # strip html tags
    text = p2.sub(tonemarkPos, text)
    text = p3.sub(tonemarkSpace, text)
    text = p4.sub(thaiDigits, text)
    text = p4.sub('', text)
    text = p7.sub('', text)
    return text

def extract_pages(html_content):
    pages = re.split(r'<div style="page-break-before:always; page-break-after:always">', html_content)
    return pages

In [6]:
for num in range(len(list_of_name)):

    inputf = open(f'./html_output/after_mapping/{list_of_name[num]}.html', 'r')
    text = inputf.read()
    inputf.close()

    text = process_text(text)


    outputf = open(f'raw_txt_output/{list_of_name[num]}.txt', 'w')
    outputf.write(text)
    outputf.close()

## Shift tone with permutation

### shift tone pupu

In [7]:

tone = [3655, 3656, 3657, 3658, 3659, 3660]
vowel = [3636, 3637, 3638, 3639, 3640, 3641, 3633]


def check_tone_vowel_sentence(new):
    last_check = thaispellcheck.check(new)
    #print(last_check)
    if "คำผิด" in last_check:
        wrong_word = re.findall(r'<คำผิด>(.*?)</คำผิด>', last_check)
        #print(wrong_word)
        for word in wrong_word:
            word_unicode = [ord(char) for char in word]
            corrected = False
            for i in range(len(word_unicode)):
                if word_unicode[i] in tone or word_unicode[i] in vowel:
                    for j in range(i):
                        if word_unicode[j] not in tone and word_unicode[j] not in vowel:
                            word_unicode[i], word_unicode[j] = word_unicode[j], word_unicode[i]
                                # เช็คคำที่แก้ไขแล้วกับ thaispellcheck
                            new_word = ''.join(chr(c) for c in word_unicode)
                           # print(new_word)
                                # print("PYTHAI:", new_word)
                            check_result = thaispellcheck.check(new_word)
                            if "คำผิด" not in check_result:
                                corrected = True
                                break
                            else:
                                    # ย้ายกลับตำแหน่งเดิม
                                word_unicode[i], word_unicode[j] = word_unicode[j], word_unicode[i]
                    if corrected:
                        break

            if corrected:
                new_word = ''.join(chr(c) for c in word_unicode)
                new = new.replace(word, new_word)
    return new                

### shift tone pKo

In [8]:
def check_tone_vowel_word(word):
    tone = [chr(3655), chr(3656), chr(3657), chr(3658), chr(3659), chr(3660)]
    vowel = [chr(3633), chr(3636), chr(3637), chr(3638), chr(3639), chr(3640), chr(3641)]

    def is_tone_or_vowel(char):
        return char in tone or char in vowel

    # Function to move each marker to all possible positions
    def move_markers(word, marker_indices):
        permutations = set()

        def swap(word_list, i, j):
            word_list[i], word_list[j] = word_list[j], word_list[i]
            return word_list

        # Create all permutations by moving each marker to every possible position
        for i, idx in enumerate(marker_indices):
            for pos in range(1,len(word)):
                if pos != idx and not is_tone_or_vowel(word[pos]):
                    new_word = list(word)
                    new_word.insert(pos, new_word.pop(idx))
                    if "คำผิด" not in thaispellcheck.check("".join(new_word)):
                        permutations.add("".join(new_word))
                        return permutations
        permutations.add("")
        return permutations
    # Find the indices of vowels and tone markers
    marker_indices = [i for i, char in enumerate(word) if is_tone_or_vowel(char)]

    # Generate permutations by moving markers
    all_permutations = set()
    all_permutations.update(move_markers(word, marker_indices))
    if list(all_permutations)[0] == "":
        return word
    else:
        return list(all_permutations)[0] 

### full shift

In [9]:
txt_path = 'raw_txt_output/'
lst_txt = [txt_path+file for file in os.listdir(txt_path)]
lst_txt

['raw_txt_output/10-ก.พ.-2566+Academic_100223_115125.txt',
 'raw_txt_output/11-ธ.ค.-2555+Academic_111212_150945.txt',
 'raw_txt_output/11-ธ.ค.-2555+Academic_111212_152818.txt',
 'raw_txt_output/11-ธ.ค.-2555+Academic_111212_154616.txt',
 'raw_txt_output/11-ธ.ค.-2555+Academic_111212_154855.txt',
 'raw_txt_output/3-ก.พ.-2565+Academic_110821_091216.txt',
 'raw_txt_output/3-ต.ค.-2566+Academic_031023_105516.txt',
 'raw_txt_output/4-มิ.ย.-2556+Academic_040613_100424.txt',
 'raw_txt_output/4-มิ.ย.-2556+Academic_040613_100623.txt',
 'raw_txt_output/9-ส.ค.-2566+Academic_130623_110407.txt']

In [14]:
txt_path = 'raw_txt_output/'
lst_txt = [txt_path+file for file in os.listdir(txt_path)]

wrong_words = []

all_lines = []

for txt_file in lst_txt:

    with open(txt_file, 'r', encoding='utf-8') as f:
        txt = f.read()
    x = 0
    for line in txt.split("\n"):
        fix_text_line = ""
        if line.strip() != "":
            new = line.replace("ำ", "า")
            if re.search(r'([ก-ฮ])ํ า', new):
                new = re.sub(r'([ก-ฮ])ํ า', r'\1ำ', new)
            if re.search(r'([ก-ฮ])่ า', new):
                new = re.sub(r'([ก-ฮ])่ า', r'\1่า', new)
            if re.search(r'([ก-ฮ])้ า', new):
                new = re.sub(r'([ก-ฮ])้ า', r'\1้า', new)
            if re.search(r'([ก-ฮ])๊ า', new):
                new = re.sub(r'([ก-ฮ])๊ า', r'\1๊า', new)
            if re.search(r'([ก-ฮ])๋ า', new):
                new = re.sub(r'([ก-ฮ])๋ า', r'\1๋า', new)
            if re.search(r'([ก-ฮ]) ำ', new):
                new = re.sub(r'([ก-ฮ]) ำ', r'\1ำ', new)
            if re.search(r'([ก-ฮ])า ง', new):
                new = re.sub(r'([ก-ฮ])า ง', r'า\1ง', new)
            if re.search(r'([ก-ฮ])า ง', new):
                new = re.sub(r'([ก-ฮ])า ง', r'า\1ง', new)
            if re.search(r'([ก-ฮ])ื อ', new):
                new = re.sub(r'([ก-ฮ])ื อ', r'\1ือ', new)
            if re.search(r'([ก-ฮ])\s้', new):
                new = re.sub(r'([ก-ฮ])\s้', r'\1้', new)
            if re.search(r'([ก-ฮ])\s่', new):
                new = re.sub(r'([ก-ฮ])\s่', r'\1่', new)
            new = new.replace(" า", "ำ")
            new = new.replace("่ื", "ื่")
            new = new.replace("้ื", "ื้")
            new = check_tone_vowel_sentence(new)
            new_line = tokenize(new)
            fix_line = []
            for text in new_line:
                # print(f'before:{text}')
                if "คำผิด" in thaispellcheck.check(text):
                    text = check_tone_vowel_word(text)
                    fix_line.append(text)
                    # print(f'after:{text}')
                    if "คำผิด" in thaispellcheck.check(text):
                        x+=1
                else:
                    fix_line.append(text)
            fix_text_line = "".join(fix_line)
        else:
            continue
        all_lines.append(fix_text_line)
    print(f"Count wrong word in {txt_file} :", x)
    all_text_lines = "\n".join(all_lines)

    with open(f'./corrected_txt_output/{txt_file.split("/")[-1]}', 'w', encoding='utf-8') as f:
        f.write(all_text_lines)

c:\Users\Lenovo\anaconda3\envs\openth\lib\site-packages\attacut\models\__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_p

Count wrong word in raw_txt_output/10-ก.พ.-2566+Academic_100223_115125.txt : 1844
Count wrong word in raw_txt_output/11-ธ.ค.-2555+Academic_111212_150945.txt : 2
Count wrong word in raw_txt_output/11-ธ.ค.-2555+Academic_111212_152818.txt : 283
Count wrong word in raw_txt_output/11-ธ.ค.-2555+Academic_111212_154616.txt : 0
Count wrong word in raw_txt_output/11-ธ.ค.-2555+Academic_111212_154855.txt : 3925
Count wrong word in raw_txt_output/3-ก.พ.-2565+Academic_110821_091216.txt : 383
Count wrong word in raw_txt_output/3-ต.ค.-2566+Academic_031023_105516.txt : 1808
Count wrong word in raw_txt_output/4-มิ.ย.-2556+Academic_040613_100424.txt : 3191
Count wrong word in raw_txt_output/4-มิ.ย.-2556+Academic_040613_100623.txt : 2222
Count wrong word in raw_txt_output/9-ส.ค.-2566+Academic_130623_110407.txt : 1604


## Concurrent

In [10]:
"""
----------------------Mapping HTML----------------------------
"""

PUA = {
    '63233': '&#3636;', # 0xf701 Sara I
    '63234': '&#3637;', # 0xf702
    '63235': '&#3638;', # 0xf703
    '63236': '&#3639;', # 0xf704
    '63237': '&#3656;', # 0xf705
    '63238': '&#3657;', # 0xf706 Mai Tho (on Po Pla)
    '63242': '&#3656;', # 0xf70a Mai Ek
    '63243': '&#3657;', # 0xf70b Mai Tho
    '63246': '&#3660;', # 0xf70e Thantakat
    '63248': '&#3633;', # 0xf710  Mai Han Akhat (on Po Pla)
    '63250': '&#3655;', # 0xf712 Mai Tai Khu (on Po Pla)
    '63251': '&#3656;', # 0xf713
    '63252': '&#3657;', # 0xf714
    '63244': '&#3658;', # ๊
    '63247': '&#3597;', #ญ
    '61482': '&#42;',   #*
    '61591': '&#8226', #•
    '63240': '&#3657', # Mai Tho
    '63241': '&#3652', #Thantakat (on Po Pla)
    '65288': '&#40;', #(
    '65289': '&#41;', #)
    '61623': '&#8226', #•
    '65309': '&#61', #=
    '65293': '&#45', #-
    '65374': '&#126', #~
}

def thaiPUA(matchobj):
    return PUA[matchobj.group(1)]

p = re.compile(r'\&\#(6\d{4,})\;')

"""
----------------------HTML to TXT----------------------------
"""

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)
    
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def tonemarkPos(matchobj):
    return matchobj.group(2) + matchobj.group(1)

def tonemarkSpace(matchobj):
    return matchobj.group(1) + matchobj.group(2)

def process_text(text):
    p2 = re.compile(r'([่้๊๋])([ัุูิีึื])') # <tonemark><vowel> -> <vowel><tonemark>
    p3 = re.compile(r'([ัุูิีึื])\s+([่้๊๋])') # <vowel><space><tonemark> -> <vowel><tonemark>
    p4 = re.compile(r'[ ]+$', re.MULTILINE) # <space>+ -> <space>
    p7 = re.compile(r'\s+(?=[ัุูิีึื])') # <space><vowel> -> <vowel>

    text = strip_tags(text)  # strip html tags
    text = p2.sub(tonemarkPos, text)
    text = p3.sub(tonemarkSpace, text)
    text = p4.sub('', text)
    text = p7.sub('', text)
    return text

def check_tone_vowel_sentence(new):
    tone = [3655, 3656, 3657, 3658, 3659, 3660]
    vowel = [3636, 3637, 3638, 3639, 3640, 3641, 3633]
    last_check = thaispellcheck.check(new)
    #print(last_check)
    if "คำผิด" in last_check:
        wrong_word = re.findall(r'<คำผิด>(.*?)</คำผิด>', last_check)
        #print(wrong_word)
        for word in wrong_word:
            word_unicode = [ord(char) for char in word]
            corrected = False
            for i in range(len(word_unicode)):
                if word_unicode[i] in tone or word_unicode[i] in vowel:
                    for j in range(i):
                        if word_unicode[j] not in tone and word_unicode[j] not in vowel:
                            word_unicode[i], word_unicode[j] = word_unicode[j], word_unicode[i]
                                # เช็คคำที่แก้ไขแล้วกับ thaispellcheck
                            new_word = ''.join(chr(c) for c in word_unicode)
                           # print(new_word)
                                # print("PYTHAI:", new_word)
                            check_result = thaispellcheck.check(new_word)
                            if "คำผิด" not in check_result:
                                corrected = True
                                break
                            else:
                                    # ย้ายกลับตำแหน่งเดิม
                                word_unicode[i], word_unicode[j] = word_unicode[j], word_unicode[i]
                    if corrected:
                        break

            if corrected:
                new_word = ''.join(chr(c) for c in word_unicode)
                new = new.replace(word, new_word)
    return new

def check_tone_vowel_word(word):
    tone = [chr(3655), chr(3656), chr(3657), chr(3658), chr(3659), chr(3660)]
    vowel = [chr(3633), chr(3636), chr(3637), chr(3638), chr(3639), chr(3640), chr(3641)]

    def is_tone_or_vowel(char):
        return char in tone or char in vowel

    # Function to move each marker to all possible positions
    def move_markers(word, marker_indices):
        permutations = set()

        def swap(word_list, i, j):
            word_list[i], word_list[j] = word_list[j], word_list[i]
            return word_list

        # Create all permutations by moving each marker to every possible position
        for i, idx in enumerate(marker_indices):
            for pos in range(1,len(word)):
                if pos != idx and not is_tone_or_vowel(word[pos]):
                    new_word = list(word)
                    new_word.insert(pos, new_word.pop(idx))
                    if "คำผิด" not in thaispellcheck.check("".join(new_word)):
                        permutations.add("".join(new_word))
                        return permutations
        permutations.add("")
        return permutations
    # Find the indices of vowels and tone markers
    marker_indices = [i for i, char in enumerate(word) if is_tone_or_vowel(char)]

    # Generate permutations by moving markers
    all_permutations = set()
    all_permutations.update(move_markers(word, marker_indices))
    if list(all_permutations)[0] == "":
        return word
    else:
        return list(all_permutations)[0] 

In [11]:
def process_pdf(pdf_name):
    """
        Convert PDF to HTML
    """
    input_format = '--input='+'pdf_example/'+pdf_name
    output_path = "./html_output/before_mapping"
    os.makedirs(output_path, exist_ok=True)
    command = ['java', '-jar', 'pdfbox-app-3.0.2.jar', 'export:text', '-html', input_format, f'--output={output_path}/{pdf_name[:-4]}.html']
    result = subprocess.run(command, capture_output=True, text=True)

    """
        Mapping PUA to Unicode
    """
    output_after_mapping = "./html_output/after_mapping/"
    os.makedirs(output_after_mapping, exist_ok=True)
    outputf = open(f'{output_after_mapping}/{pdf_name[:-4]}.html', 'w')
    inputf = open(f'{output_path}/{pdf_name[:-4]}.html', 'r')
    for line in inputf:
        text = p.sub(thaiPUA, line)
        outputf.writelines(html.unescape(text))
    inputf.close()
    outputf.close()

    """
        Extract text from HTML
    """
    inputf = open(f'{output_after_mapping}/{pdf_name[:-4]}.html', 'r')
    text = inputf.read()
    inputf.close()

    text = process_text(text)

    os.makedirs('./raw_txt_output', exist_ok=True)
    outputf = open(f'raw_txt_output/{pdf_name[:-4]}.txt', 'w')
    outputf.write(text)
    outputf.close()

    """
        Check tone and vowel
    """
    all_lines = []
    x = 0
    for line in text.split("\n"):
        fix_text_line = ""
        if line.strip() != "":
            new = line.replace("ำ", "า")
            if re.search(r'([ก-ฮ])ํ า', new):
                new = re.sub(r'([ก-ฮ])ํ า', r'\1ำ', new)
            if re.search(r'([ก-ฮ])่ า', new):
                new = re.sub(r'([ก-ฮ])่ า', r'\1่า', new)
            if re.search(r'([ก-ฮ])้ า', new):
                new = re.sub(r'([ก-ฮ])้ า', r'\1้า', new)
            if re.search(r'([ก-ฮ])๊ า', new):
                new = re.sub(r'([ก-ฮ])๊ า', r'\1๊า', new)
            if re.search(r'([ก-ฮ])๋ า', new):
                new = re.sub(r'([ก-ฮ])๋ า', r'\1๋า', new)
            if re.search(r'([ก-ฮ]) ำ', new):
                new = re.sub(r'([ก-ฮ]) ำ', r'\1ำ', new)
            if re.search(r'([ก-ฮ])า ง', new):
                new = re.sub(r'([ก-ฮ])า ง', r'า\1ง', new)
            if re.search(r'([ก-ฮ])า ง', new):
                new = re.sub(r'([ก-ฮ])า ง', r'า\1ง', new)
            if re.search(r'([ก-ฮ])ื อ', new):
                new = re.sub(r'([ก-ฮ])ื อ', r'\1ือ', new)
            if re.search(r'([ก-ฮ])\s้', new):
                new = re.sub(r'([ก-ฮ])\s้', r'\1้', new)
            if re.search(r'([ก-ฮ])\s่', new):
                new = re.sub(r'([ก-ฮ])\s่', r'\1่', new)
            new = new.replace(" า", "ำ")
            new = new.replace("่ื", "ื่")
            new = new.replace("้ื", "ื้")
            new = check_tone_vowel_sentence(new)
            new_line = tokenize(new)
            fix_line = []
            for text in new_line:
                    # print(f'before:{text}')
                    if "คำผิด" in thaispellcheck.check(text):
                        text = check_tone_vowel_word(text)
                        fix_line.append(text)
                        # print(f'after:{text}')
                        if "คำผิด" in thaispellcheck.check(text):
                            x+=1
                    else:
                        fix_line.append(text)
            fix_text_line = "".join(fix_line)
        else:
            continue
        all_lines.append(fix_text_line)
    print(f"{pdf_name} : Count wrong word :", x)
    all_text_lines = "\n".join(all_lines)

    os.makedirs('./corrected_txt_output', exist_ok=True)
    with open(f'./corrected_txt_output/{pdf_name[:-4]}.txt', 'w', encoding='utf-8') as f:
        f.write(all_text_lines)
    return {pdf_name: x}

In [13]:
process_pdf('9-ส.ค.-2566+Academic_130623_110407.pdf')

Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte
c:\Users\Lenovo\anaconda3\envs\openth\lib\site-packages\attacut\models\__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code durin

9-ส.ค.-2566+Academic_130623_110407.pdf : Count wrong word : 1604


{'9-ส.ค.-2566+Academic_130623_110407.pdf': 1604}

### Thread

In [4]:
from concurrent.futures import ThreadPoolExecutor

pdf_list = os.listdir('./pdf_example')

# Use ThreadPoolExecutor to process PDFs concurrently
max_workers = min(4, len(pdf_list))  # Use at most 4 threads or the number of PDFs
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_pdf, pdf_name) for pdf_name in pdf_list]

    # Optionally, wait for all tasks to complete and handle exceptions
    for future in futures:
        try:
            future.result()  # Wait for each task to complete
        except Exception as e:
            print(f"Error processing PDF: {e}")

11-ธ.ค.-2555+Academic_111212_154616.pdf : Count wrong word : 0


Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte
c:\Users\Lenovo\anaconda3\envs\openth\lib\site-packages\attacut\models\__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code durin

11-ธ.ค.-2555+Academic_111212_152818.pdf : Count wrong word : 283
3-ก.พ.-2565+Academic_110821_091216.pdf : Count wrong word : 383
11-ธ.ค.-2555+Academic_111212_150945.pdf : Count wrong word : 2
11-ธ.ค.-2555+Academic_111212_154855.pdf : Count wrong word : 3925
4-มิ.ย.-2556+Academic_040613_100623.pdf : Count wrong word : 2222


Exception in thread Thread-24:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte


4-มิ.ย.-2556+Academic_040613_100424.pdf : Count wrong word : 3191
10-ก.พ.-2566+Academic_100223_115125.pdf : Count wrong word : 1844
3-ต.ค.-2566+Academic_031023_105516.pdf : Count wrong word : 1808
Error processing PDF: '65288'


### Process

In [5]:
from concurrent.futures import ProcessPoolExecutor

pdf_list = os.listdir('./pdf_example')

# Use ProcessPoolExecutor to process PDFs concurrently
max_workers = min(4, len(pdf_list))  # Use at most 4 processes or the number of PDFs
with ProcessPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_pdf, pdf_name) for pdf_name in pdf_list]

    # Optionally, wait for all tasks to complete and handle exceptions
    for future in futures:
        try:
            future.result()  # Wait for each task to complete
        except Exception as e:
            print(f"Error processing PDF: {e}")

Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing PDF: A process in the process pool was terminat

### Thread in process each line

In [12]:
def process_line(line):
    fix_text_line = ""
    x = 0
    if line.strip() != "":
        new = line.replace("ำ", "า")
        if re.search(r'([ก-ฮ])ํ า', new):
            new = re.sub(r'([ก-ฮ])ํ า', r'\1ำ', new)
        if re.search(r'([ก-ฮ])่ า', new):
            new = re.sub(r'([ก-ฮ])่ า', r'\1่า', new)
        if re.search(r'([ก-ฮ])้ า', new):
            new = re.sub(r'([ก-ฮ])้ า', r'\1้า', new)
        if re.search(r'([ก-ฮ])๊ า', new):
            new = re.sub(r'([ก-ฮ])๊ า', r'\1๊า', new)
        if re.search(r'([ก-ฮ])๋ า', new):
            new = re.sub(r'([ก-ฮ])๋ า', r'\1๋า', new)
        if re.search(r'([ก-ฮ]) ำ', new):
            new = re.sub(r'([ก-ฮ]) ำ', r'\1ำ', new)
        if re.search(r'([ก-ฮ])า ง', new):
            new = re.sub(r'([ก-ฮ])า ง', r'า\1ง', new)
        if re.search(r'([ก-ฮ])ื อ', new):
            new = re.sub(r'([ก-ฮ])ื อ', r'\1ือ', new)
        if re.search(r'([ก-ฮ])\s้', new):
            new = re.sub(r'([ก-ฮ])\s้', r'\1้', new)
        if re.search(r'([ก-ฮ])\s่', new):
            new = re.sub(r'([ก-ฮ])\s่', r'\1่', new)
        new = new.replace(" า", "ำ")
        new = new.replace("่ื", "ื่")
        new = new.replace("้ื", "ื้")
        new = check_tone_vowel_sentence(new)
        new_line = tokenize(new)
        fix_line = []
        for text in new_line:
            if "คำผิด" in thaispellcheck.check(text):
                text = check_tone_vowel_word(text)
                fix_line.append(text)
                if "คำผิด" in thaispellcheck.check(text):
                    x += 1
            else:
                fix_line.append(text)
        fix_text_line = "".join(fix_line)
    return fix_text_line, x

def process_all_text(text):
    all_lines = []
    total_wrong_words = 0

    with ThreadPoolExecutor(max_workers=4) as executor:
        # Process lines in parallel
        results = list(executor.map(process_line, text.split("\n")))

    # Collect results
    for fix_text_line, wrong_count in results:
        all_lines.append(fix_text_line)
        total_wrong_words += wrong_count

    print(f"Count of wrong words: {total_wrong_words}")
    all_text_lines = "\n".join(all_lines)
    return all_text_lines, total_wrong_words

def process_pdf(pdf_name):
    """
        Convert PDF to HTML
    """
    input_format = '--input='+'pdf_example/'+pdf_name
    output_path = "./html_output/before_mapping"
    os.makedirs(output_path, exist_ok=True)
    command = ['java', '-jar', 'pdfbox-app-3.0.2.jar', 'export:text', '-html', input_format, f'--output={output_path}/{pdf_name[:-4]}.html']
    result = subprocess.run(command, capture_output=True, text=True)

    """
        Mapping PUA to Unicode
    """
    output_after_mapping = "./html_output/after_mapping/"
    os.makedirs(output_after_mapping, exist_ok=True)
    outputf = open(f'{output_after_mapping}/{pdf_name[:-4]}.html', 'w')
    inputf = open(f'{output_path}/{pdf_name[:-4]}.html', 'r')
    for line in inputf:
        text = p.sub(thaiPUA, line)
        outputf.writelines(html.unescape(text))
    inputf.close()
    outputf.close()

    """
        Extract text from HTML
    """
    inputf = open(f'{output_after_mapping}/{pdf_name[:-4]}.html', 'r')
    text = inputf.read()
    inputf.close()

    text = process_text(text)

    os.makedirs('./raw_txt_output', exist_ok=True)
    outputf = open(f'raw_txt_output/{pdf_name[:-4]}.txt', 'w')
    outputf.write(text)
    outputf.close()

    """
        Check tone and vowel
    """
    all_text_lines, x = process_all_text(text)

    os.makedirs('./corrected_txt_output', exist_ok=True)
    with open(f'./corrected_txt_output/{pdf_name[:-4]}.txt', 'w', encoding='utf-8') as f:
        f.write(all_text_lines)
    return pdf_name, x

In [13]:
pdf_list = os.listdir('./pdf_example')
for pdf_name in pdf_list:
    result, x = process_pdf(pdf_name)
    print(f"{pdf_name} : Count wrong word : {x}")

c:\Users\Lenovo\anaconda3\envs\openth\lib\site-packages\attacut\models\__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_p

Count of wrong words: 1844
10-ก.พ.-2566+Academic_100223_115125.pdf : Count wrong word : 1844
Count of wrong words: 2
11-ธ.ค.-2555+Academic_111212_150945.pdf : Count wrong word : 2


Exception in thread Thread-20:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte


Count of wrong words: 283
11-ธ.ค.-2555+Academic_111212_152818.pdf : Count wrong word : 283
Count of wrong words: 0
11-ธ.ค.-2555+Academic_111212_154616.pdf : Count wrong word : 0
Count of wrong words: 3925
11-ธ.ค.-2555+Academic_111212_154855.pdf : Count wrong word : 3925
Count of wrong words: 383
3-ก.พ.-2565+Academic_110821_091216.pdf : Count wrong word : 383
Count of wrong words: 1808
3-ต.ค.-2566+Academic_031023_105516.pdf : Count wrong word : 1808
Count of wrong words: 3191
4-มิ.ย.-2556+Academic_040613_100424.pdf : Count wrong word : 3191
Count of wrong words: 2222
4-มิ.ย.-2556+Academic_040613_100623.pdf : Count wrong word : 2222


Exception in thread Thread-34:
Traceback (most recent call last):
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\subprocess.py", line 1495, in _readerthread
    buffer.append(fh.read())
  File "c:\Users\Lenovo\anaconda3\envs\openth\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 0: invalid continuation byte


KeyError: '65288'